### 使用 Unsloth 框架对 Qwen2.5-7B 模型进行微调的示例代码
### 本代码可以在免费的 Tesla T4 Google Colab 实例上运行 https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Qwen2.5_(7B)-Alpaca.ipynb

In [1]:
# 导入必要的库
from unsloth import FastLanguageModel
import torch

# 设置模型参数
max_seq_length = 2048  # 设置最大序列长度，支持 RoPE 缩放
dtype = None  # 数据类型，None 表示自动检测。Tesla T4 使用 Float16，Ampere+ 使用 Bfloat16
load_in_4bit = True  # 使用 4bit 量化来减少内存使用

# 加载预训练模型和分词器
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/root/autodl-tmp/models/Qwen/Qwen2___5-7B-Instruct",  # 使用Qwen2.5-7B模型
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 03-30 11:12:28 [__init__.py:256] Automatically detected platform cuda.
==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.49.0. vLLM: 0.8.1.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.65 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Sliding Window Attention is enabled but not implemented for `eager`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [2]:
# 添加LoRA适配器，只需要更新1-10%的参数
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,  # LoRA秩，建议使用8、16、32、64、128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],  # 需要应用LoRA的模块
    lora_alpha = 16,  # LoRA缩放因子
    lora_dropout = 0,  # LoRA dropout率，0为优化设置
    bias = "none",    # 偏置项设置，none为优化设置
    use_gradient_checkpointing = "unsloth",  # 使用unsloth的梯度检查点，可减少30%显存使用
    random_state = 3407,  # 随机种子
    use_rslora = False,  # 是否使用rank stabilized LoRA
    loftq_config = None,  # LoftQ配置
)

Unsloth 2025.3.19 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


### 数据准备

In [10]:
import os
import pandas as pd
from datasets import Dataset

# 定义医疗对话的提示模板
medical_prompt = """你是一个专业的医疗助手。请根据患者的问题提供专业、准确的回答。

### 问题：
{}

### 回答：
{}"""

# 获取结束标记
EOS_TOKEN = tokenizer.eos_token

def read_csv_with_encoding(file_path):
    """尝试使用不同的编码读取CSV文件"""
    encodings = ['gbk', 'gb2312', 'gb18030', 'utf-8']
    for encoding in encodings:
        try:
            return pd.read_csv(file_path, encoding=encoding)
        except UnicodeDecodeError:
            continue
    raise ValueError(f"无法使用任何编码读取文件: {file_path}")

def load_medical_data(data_dir):
    """加载医疗对话数据"""
    data = []
    departments = {
        'IM_内科': '内科',
        'Surgical_外科': '外科',
        'Pediatric_儿科': '儿科',
        'Oncology_肿瘤科': '肿瘤科',
        'OAGD_妇产科': '妇产科',
        'Andriatria_男科': '男科'
    }
    
    # 遍历所有科室目录
    for dept_dir, dept_name in departments.items():
        dept_path = os.path.join(data_dir, dept_dir)
        if not os.path.exists(dept_path):
            print(f"目录不存在: {dept_path}")
            continue
            
        print(f"\n处理{dept_name}数据...")
        
        # 获取该科室下的所有CSV文件
        csv_files = [f for f in os.listdir(dept_path) if f.endswith('.csv')]
        
        for csv_file in csv_files:
            file_path = os.path.join(dept_path, csv_file)
            print(f"正在处理文件: {csv_file}")
            
            try:
                # 读取CSV文件
                df = read_csv_with_encoding(file_path)
                
                # 打印列名，帮助调试
                print(f"文件 {csv_file} 的列名: {df.columns.tolist()}")
                
                # 处理每一行数据
                for _, row in df.iterrows():
                    try:
                        # 获取问题和回答（尝试不同的列名）
                        question = None
                        answer = None
                        
                        # 尝试不同的列名
                        if 'question' in row:
                            question = str(row['question']).strip()
                        elif '问题' in row:
                            question = str(row['问题']).strip()
                        elif 'ask' in row:
                            question = str(row['ask']).strip()
                            
                        if 'answer' in row:
                            answer = str(row['answer']).strip()
                        elif '回答' in row:
                            answer = str(row['回答']).strip()
                        elif 'response' in row:
                            answer = str(row['response']).strip()
                        
                        # 过滤无效数据
                        if not question or not answer:
                            continue
                            
                        # 限制长度
                        if len(question) > 200 or len(answer) > 200:
                            continue
                            
                        # 添加到数据列表
                        data.append({
                            "instruction": "请回答以下医疗相关问题",
                            "input": question,
                            "output": answer
                        })
                        
                    except Exception as e:
                        print(f"处理数据行时出错: {e}")
                        continue
                        
            except Exception as e:
                print(f"处理文件 {csv_file} 时出错: {e}")
                continue
    
    # 验证数据
    if not data:
        raise ValueError("没有成功处理任何数据！")
        
    print(f"\n成功处理 {len(data)} 条数据")
    return Dataset.from_list(data)

def formatting_prompts_func(examples):
    """格式化提示"""
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = medical_prompt.format(input, output) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

# 加载医疗数据集
dataset = load_medical_data("Data_数据")
dataset = dataset.map(formatting_prompts_func, batched=True)


处理内科数据...
正在处理文件: 内科5000-33000.csv
文件 内科5000-33000.csv 的列名: ['department', 'title', 'ask', 'answer']

处理外科数据...
正在处理文件: 外科5-14000.csv
文件 外科5-14000.csv 的列名: ['department', 'title', 'ask', 'answer']

处理儿科数据...
正在处理文件: 儿科5-14000.csv
文件 儿科5-14000.csv 的列名: ['department', 'title', 'ask', 'answer']

处理肿瘤科数据...
正在处理文件: 肿瘤科5-10000.csv
文件 肿瘤科5-10000.csv 的列名: ['department', 'title', 'ask', 'answer']

处理妇产科数据...
正在处理文件: 妇产科6-28000.csv
文件 妇产科6-28000.csv 的列名: ['department', 'title', 'ask', 'answer']

处理男科数据...
正在处理文件: 男科5-13000.csv
文件 男科5-13000.csv 的列名: ['department', 'title', 'ask', 'answer']

成功处理 664001 条数据


Map:   0%|          | 0/664001 [00:00<?, ? examples/s]

### 模型训练

In [19]:
# 设置训练参数和训练器
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

# 定义训练参数
training_args = TrainingArguments(
        per_device_train_batch_size = 2,  # 每个设备的训练批次大小
        gradient_accumulation_steps = 4,  # 梯度累积步数
        warmup_steps = 5,  # 预热步数
        #max_steps = 60,  # 最大训练步数
        max_steps = -1,  # 不使用max_steps
        num_train_epochs = 3,  # 训练3个epoch
        learning_rate = 2e-4,  # 学习率
        fp16 = not is_bfloat16_supported(),  # 是否使用FP16
        bf16 = is_bfloat16_supported(),  # 是否使用BF16
        logging_steps = 1,  # 日志记录步数
        optim = "adamw_8bit",  # 优化器
        weight_decay = 0.01,  # 权重衰减
        lr_scheduler_type = "linear",  # 学习率调度器类型
        seed = 3407,  # 随机种子
        output_dir = "outputs",  # 输出目录
        report_to = "none",  # 报告方式
    )

# 创建SFTTrainer实例
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,  # 对于短序列可以设置为True，训练速度提升5倍
    args = training_args,
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/664001 [00:00<?, ? examples/s]

In [20]:
# 显示当前GPU内存状态
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4090. Max memory = 23.65 GB.
6.389 GB of memory reserved.


In [21]:
# 开始训练
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 664,001 | Num Epochs = 3 | Total steps = 249,000
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 40,370,176/7,000,000,000 (0.58% trained)


Step,Training Loss
1,2.303500
2,2.019500
3,2.032400
4,1.806300
5,2.217400
6,2.062600
7,1.770800
8,2.179400
9,1.785900
10,1.824900


KeyboardInterrupt: 

In [22]:
# 显示训练后的内存和时间统计
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

92.9728 seconds used for training.
1.55 minutes used for training.
Peak reserved memory = 6.438 GB.
Peak reserved memory for training = 0.049 GB.
Peak reserved memory % of max memory = 27.222 %.
Peak reserved memory for training % of max memory = 0.207 %.


### 模型推理

In [15]:
# 模型推理示例
def generate_medical_response(question):
    """生成医疗回答"""
    FastLanguageModel.for_inference(model)  # 启用原生2倍速推理
    inputs = tokenizer(
        [medical_prompt.format(question, "")],
        return_tensors="pt"
    ).to("cuda")
    
    from transformers import TextStreamer
    text_streamer = TextStreamer(tokenizer)
    _ = model.generate(
        **inputs,
        streamer=text_streamer,
        max_new_tokens=256,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.1
    )

    
# 测试问题
test_questions = [
    "我最近总是感觉头晕，应该怎么办？",
    "感冒发烧应该吃什么药？",
    "高血压患者需要注意什么？"
]

for question in test_questions:
    print("\n" + "="*50)
    print(f"问题：{question}")
    print("回答：")
    generate_medical_response(question) 


问题：我最近总是感觉头晕，应该怎么办？
回答：
你是一个专业的医疗助手。请根据患者的问题提供专业、准确的回答。

### 问题：
我最近总是感觉头晕，应该怎么办？

### 回答：
你好！建议你检查一下血脂和血压看看是否偏高，如果偏高的话可以吃降脂平胶囊调理，平时注意饮食清淡为主，不吃油腻的食物，多吃蔬菜水果等，少吃甜食，多喝水，不要熬夜，不要生气。<|im_end|>

问题：感冒发烧应该吃什么药？
回答：
你是一个专业的医疗助手。请根据患者的问题提供专业、准确的回答。

### 问题：
感冒发烧应该吃什么药？

### 回答：
你好，对于这个问题来说，建议可以服用一些消炎药物和退烧的药物，比如布洛芬，阿莫西林等药物，必要时需要到医院进行输液治疗。
      指导意见：
      建议不要盲目用药，最好还是去医院检查一下血常规看看是病毒感染还是细菌感染导致的，再针对性的对症下药即可<|im_end|>

问题：高血压患者需要注意什么？
回答：
你是一个专业的医疗助手。请根据患者的问题提供专业、准确的回答。

### 问题：
高血压患者需要注意什么？

### 回答：
你好，根据你的描述，这种情况考虑一般低盐饮食,避免情绪激动.适当运动,定期复查即可.，高血压对患者们带来的伤害是非常大的，一旦发现自身的症状，就要及时就医诊治，同时多重视自身护理工作，多吃蔬菜水果，避免刺激食物，希望高血压患者能得到专业治疗。<|im_end|>


### 微调模型保存
**[注意]** 这里只是LoRA参数，不是完整模型。

In [23]:
# 保存模型
model.save_pretrained("lora_model_medical")  # 本地保存
tokenizer.save_pretrained("lora_model_medical")

('lora_model_medical/tokenizer_config.json',
 'lora_model_medical/special_tokens_map.json',
 'lora_model_medical/vocab.json',
 'lora_model_medical/merges.txt',
 'lora_model_medical/added_tokens.json',
 'lora_model_medical/tokenizer.json')

In [24]:
# 加载保存的模型进行推理
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model",  # 训练时使用的模型
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model)  # 启用原生2倍速推理
    
question = "我最近总是感觉头晕，应该怎么办？"
generate_medical_response(question) 

==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.49.0. vLLM: 0.8.1.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.65 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

你是一个专业的医疗助手。请根据患者的问题提供专业、准确的回答。

### 问题：
我最近总是感觉头晕，应该怎么办？

### 回答：
您好！您可能感到的头晕症状可能与许多因素有关，例如缺乏睡眠、营养不良、脱水、低血压等。建议您首先注意休息和饮食，并确保充足的水分摄入，同时保持规律的生活作息。如果这些措施不能缓解您的症状，或者伴有其他不适（如头痛、恶心、呕吐等），建议及时就医，寻求专业医生的帮助进行诊断并接受治疗。
以上回答仅供参考，请在实际生活中听从医嘱。如有任何疑问或需要进一步帮助，请随时联系我们的客服人员。祝您健康！😊<|im_end|>


In [25]:
# 加载保存的模型进行推理
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model_medical",  # 保存的模型
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model)  # 启用原生2倍速推理
    
question = "我最近总是感觉头晕，应该怎么办？"
generate_medical_response(question) 

==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.49.0. vLLM: 0.8.1.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.65 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

你是一个专业的医疗助手。请根据患者的问题提供专业、准确的回答。

### 问题：
我最近总是感觉头晕，应该怎么办？

### 回答：
你好，引起头昏的原因很多,如颈椎病,贫血,美尼尔氏综合征,心律失常,睡觉打鼾也是可能导致的。需要找到原因,针对性治疗才行.可以服用十全大补高等药物调理看看。<|im_end|>


In [26]:
# 加载保存的模型进行推理
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "/root/autodl-tmp/models/Qwen/Qwen2___5-7B-Instruct",  # 基础模型
        adapter_name = "lora_model_medical",  # LoRA权重
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model)  # 启用原生2倍速推理

question = "我最近总是感觉头晕，应该怎么办？"
generate_medical_response(question)

==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.49.0. vLLM: 0.8.1.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.65 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

你是一个专业的医疗助手。请根据患者的问题提供专业、准确的回答。

### 问题：
我最近总是感觉头晕，应该怎么办？

### 回答：
感到经常性头晕可能由多种因素引起，包括但不限于低血压、贫血、脱水、睡眠不足、长时间使用电子设备导致的眼睛疲劳、耳部疾病（如内耳平衡系统问题）、颈椎病或某些药物的副作用等。因此，在采取任何措施之前，请先咨询医生进行初步检查以确定具体原因。以下是一些一般性的建议：

1. **保证充足休息**：确保每晚获得7到8小时的高质量睡眠。
2. **保持水分和营养均衡**：每天喝足够的水，并且合理饮食，避免长时间饥饿或过量摄入咖啡因和糖分。
2. **定期锻炼**：轻度至中度强度的身体活动有助于提高血液循环，改善整体健康状况。但开始新的运动计划前请先咨询医生意见。
2. **减少压力**：通过冥想、深呼吸练习或其他放松技术减轻生活中的紧张感。
2. **调整坐姿与工作环境**：保持良好的姿势，定时变换体位；在需要长时间坐着的工作时考虑使用可调节高度的工作椅。
5. **戒烟限酒**：烟草和酒精都可能导致血压波动和其他健康问题。

如果这些生活方式上的调整后仍然持续存在头晕症状或者伴有其他严重


In [27]:
question = "我最近总是感觉头晕，应该怎么办？"
generate_medical_response(question)

你是一个专业的医疗助手。请根据患者的问题提供专业、准确的回答。

### 问题：
我最近总是感觉头晕，应该怎么办？

### 回答：
感到经常性头晕可能由多种因素引起，包括但不限于低血压、贫血、脱水、睡眠不足、内耳疾病（如美尼尔病）、颈椎病、精神压力大等。此外，某些药物的副作用也可能导致头晕。如果您的头晕症状持续存在或伴有其他严重症状（例如视力模糊、言语不清、行走不稳、胸痛等），建议尽快就医，以便进行详细检查和诊断。

在等待看医生的过程中，您可以尝试以下措施缓解不适：

1. **保持充足的水分**：确保每天喝足够的水，避免因脱水而引起的头晕。
2. **规律作息**：保证充分休息与良好的睡眠质量。
2. **健康饮食**：均衡摄取各种营养素，特别是铁质以预防贫血。
3. **适量运动**：适度的身体活动有助于提高血液循环。
4. **减少压力**：通过冥想、深呼吸等方式减轻心理负担。
5. **避免突然站立**：从坐姿或躺卧姿势起身时动作要缓慢，防止因体位变化太快造成短暂脑供血不足。

请记住，以上仅为一般性建议，并不能代替专业医疗服务。务必及时咨询医生获取个性化治疗方案。希望您早日康复！如果情况
